# How to fit Parametric Forecasts to Quantile Forecasts

In [1]:
from create_parametric_forecasts import ParametricForecasts
import pandas as pd
import numpy as np

In [2]:
#path_to_quantile_fc = 'C:\\Users\\fh6281\\PycharmProjects\\GermanBuildingDate\\02_forecast\\storage_quantile_fc\\file_fc_PatchTST_SFH3_2025-07-16_13-47-24_freq15.csv'
path_to_quantile_fc = 'C:\\Users\\fh6281\\PycharmProjects\\GermanBuildingDate\\02_forecast\\storage_quantile_fc\\file_fc_AutoKAN_SFH3_2025-07-25_14-18-05_freq15.csv'
#path_to_quantile_fc = 'C:\\Users\\fh6281\\PycharmProjects\\GermanBuildingDate\\02_forecast\\storage_quantile_fc\\file_fc_AutoKAN_SFH3_short.csv'

pf = ParametricForecasts()
pf.load_quantile_forecasts(path_to_quantile_fc, timerange=['2020-05-13 06:15:00+00:00', '2020-05-16 06:30:00+00:00'])  # timerange=None

In [3]:
pf.sort_quantiles()
pf.quantile_forecasts

Sorting quantiles in ascending order…


0.01       0.02  \
time_fc_created           timestamp                                         
2020-05-13 06:15:00+00:00 2020-05-13 06:15:00+00:00 -2684.0806 -2380.6167   
                          2020-05-13 06:30:00+00:00 -2890.3403 -2645.7670   
                          2020-05-13 06:45:00+00:00 -3379.8320 -3073.0806   
                          2020-05-13 07:00:00+00:00 -3798.5127 -3586.4620   
                          2020-05-13 07:15:00+00:00 -4161.5170 -3856.0190   
...                                                        ...        ...   
2020-05-16 06:30:00+00:00 2020-05-17 05:15:00+00:00 -2176.2993 -1763.0284   
                          2020-05-17 05:30:00+00:00 -2629.8494 -2124.0535   
                          2020-05-17 05:45:00+00:00 -2927.5437 -2549.7250   
                          2020-05-17 06:00:00+00:00 -3637.5405 -3033.8025   
                          2020-05-17 06:15:00+00:00 -3872.3574 -3574.5366   

                                                          0.03       0.04  \
time_fc_created           timestamp                                         
2020-05-13 06:15:00+00:00 2020-05-13 06:15:00+00:00 -2115.7070 -1970.4945   
                          2020-05-13 06:30:00+00:00 -2276.6685 -2268.3423   
                          2020-05-13 06:45:00+00:00 -2675.7036 -2596.2617   
                          2020-05-13 07:00:00+00:00 -3106.3047 -3077.5063   
                          2020-05-13 07:15:00+00:00 -3512.1553 -3324.5170   
...                                                        ...        ...   
2020-05-16 06:30:00+00:00 2020-05-17 05:15:00+00:00 -1502.3555 -1495.9194   
                          2020-05-17 05:30:00+00:00 -1982.0952 -1897.4148   
                          2020-05-17 05:45:00+00:00 -2442.8086 -2268.9610   
                          2020-05-17 06:00:00+00:00 -2900.4316 -2763.6128   
                          2020-05-17 06:15:00+00:00 -3224.0425 -3065.5312   

                                                          0.05       0.06  \
time_fc_created           timestamp                                         
2020-05-13 06:15:00+00:00 2020-05-13 06:15:00+00:00 -1850.6685 -1785.6593   
                          2020-05-13 06:30:00+00:00 -2225.6104 -2129.3384   
                          2020-05-13 06:45:00+00:00 -2424.3540 -2416.5903   
                          2020-05-13 07:00:00+00:00 -2880.1792 -2829.5820   
                          2020-05-13 07:15:00+00:00 -3149.9834 -3061.5996   
...                                                        ...        ...   
2020-05-16 06:30:00+00:00 2020-05-17 05:15:00+00:00 -1388.5994 -1306.5469   
                          2020-05-17 05:30:00+00:00 -1883.4660 -1836.4832   
                          2020-05-17 05:45:00+00:00 -2209.2778 -2118.2270   
                          2020-05-17 06:00:00+00:00 -2559.7363 -2494.2632   
                          2020-05-17 06:15:00+00:00 -2848.7178 -2799.9204   

                                                          0.07       0.08  \
time_fc_created           timestamp                                         
2020-05-13 06:15:00+00:00 2020-05-13 06:15:00+00:00 -1753.3043 -1609.5797   
                          2020-05-13 06:30:00+00:00 -2065.7014 -2003.8525   
                          2020-05-13 06:45:00+00:00 -2366.4646 -2311.3398   
                          2020-05-13 07:00:00+00:00 -2814.6567 -2690.3096   
                          2020-05-13 07:15:00+00:00 -2986.1826 -2969.2050   
...                                                        ...        ...   
2020-05-16 06:30:00+00:00 2020-05-17 05:15:00+00:00 -1227.0090 -1213.9059   
                          2020-05-17 05:30:00+00:00 -1721.0918 -1533.7420   
                          2020-05-17 05:45:00+00:00 -1996.6250 -1902.7389   
                          2020-05-17 06:00:00+00:00 -2387.9434 -2354.7540   
                          2020-05-17 06:15:00+00:00 -2746.7751 -2718.7754   

                                                  

In [4]:
pf.fit_distribution('sum2gaussian')

Progress: 100% (27840/27840)

In [5]:
pf.store_parametric_forecasts()  # creation_time=True

Parametric forecasts saved to C:\Users\fh6281\PycharmProjects\GermanBuildingDate\02_forecast\storage_param_fc\file_fc_parametric_AutoKAN_SFH3_2025-07-25_14-18-05_freq15.csv


# Playground
